In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import string
from nltk.tokenize import sent_tokenize, word_tokenize   
from nltk.tokenize import WordPunctTokenizer
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report


Lets take a look at the train and test dataset

In [ ]:
train_df =pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_df=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
insincere=train_df[['qid','question_text']].where(train_df['target']==1 & ~pd.isnull(train_df['question_text']))
insincere =insincere.dropna()

sincere =train_df[['qid','question_text']].where(train_df['target']==0 & ~ pd.isnull(train_df['question_text']))
sincere= sincere.dropna()

print (f"Total questions {train_df.shape[0]}")
print(f"Total insincere questions {insincere.shape[0]}")
print(f"Total sincere questions {sincere.shape[0]}")
train_df.tail()

Lets clean the question_text now

In [ ]:
punc = WordPunctTokenizer()
lemmatizer = WordNetLemmatizer() 
stop = stopwords.words('english')
exclude = set(string.punctuation)

def clean_text(text):
    word_tokens = (word_tokenize(text))
    remove_stop=[w.lower() for w in word_tokens if w.lower() not in stop]
    remove_punct=[c for c in remove_stop if c not in exclude and len(c)>3]
    clean =" ".join([re.sub(r'[^a-zA-Z0-9]','',i) for i in remove_punct ])
    lemma=lemmatizer.lemmatize(clean)
    return lemma
insincere['clean_question']=insincere['question_text'].map(clean_text)


In [ ]:
insincere['clean_question']
train_df['clean_question'] = insincere['clean_question']
train_df.head(10)

In [ ]:
# lets find out the total no of words 
word_freq={}
for w in insincere['clean_question'].values.tolist():
    for word in w.split(" "):
        if word  in word_freq.keys():
            word_freq[word]+=1
        else:
            word_freq[word]=1
print (len(word_freq))
lists = sorted(word_freq.items())


words_df=pd.DataFrame(lists,columns=['words','frequency']).sort_values(by='frequency',ascending=False)
words_df.set_index('words')[:50].plot(kind='bar',figsize=(20,10),title='Frequency Dist For Top 20 Words');

Using TFIDF vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5, strip_accents='unicode',lowercase =True, 
                        stop_words = 'english')
train_vectorized = vectorizer.fit_transform(train_df.question_text.values)
test_vectorized = vectorizer.fit_transform(test_df.question_text.values)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_vectorized, 
            train_df.target.values, test_size=0.1, stratify = train_df.target.values)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)

print (y_pred)

